In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold, cross_val_score
import time
import datetime

In [2]:
features = pd.read_csv('./features.csv', index_col='match_id')
target = features.radiant_win  # целевая переменная, т.к. мы пытаемся предсказать победителя

features = features.drop(["duration", "radiant_win",  # удаляем признаки, связанные с окончанием игры
                          "tower_status_radiant",
                          "tower_status_dire",
                         "barracks_status_radiant",
                          "barracks_status_dire"], axis=1)

### Подход 1: градиентный бустинг "в лоб"

#### Найдём признаки, информация о которых имеется не в каждом матче.

In [3]:
number_of_matches = features.shape[0]
columns_full_check = np.array(features.count() == number_of_matches)
not_full = np.array(features.columns[columns_full_check == False])
for col in not_full:
    print(col)

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


Все признаки связанные с **"first_blood"** имеют пропуски, т.к. в некоторых матчах при довольно аккуратной игре обеих из
команд первая кровь может пролиться позднее, чем через 5 минут.

Отсутствие **"bottle"** объясняется предпочтением закупки отдельных игроков, а также зависит от выбора героев.

Что же касается отсутствия **courier**-ов и **ward**-ов, то это прямой признак непрофессионализма игроков. Возможно, речь идёт о матчах
с низким приоритетом.

In [4]:
for col in not_full:
    features[col].fillna(value=0, inplace=True)  # заменяем в этих колонках отсутствующие значения на 0

### Весь наш анализ проводится с целью как можно точнее научиться предсказывать победителя в матче. А значит, целевой переменной является столбец **"radiant_win"**

In [5]:
# Далее оценим качество градиентного бустинга с помощью кросс-валидации по 5 блокам
kf = KFold(n=number_of_matches, n_folds=5, shuffle=True)
for n in [10, 20, 30, 40]:
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n, learning_rate=0.5)  # n_estimators = 10, 20, 30, 40, 60
    clf.fit(features, target)
    scores = cross_val_score(clf, features, target, cv=kf, scoring="roc_auc")
    mean_score = scores.mean()
    print("Количество деревьев =", n)
    print('Кросс-валидация проведена за:', datetime.datetime.now() - start_time)  # 0:03:19.762472
    print("Качество:", mean_score)  # 0.641047001954 (при стандартном max_depth=3 и learning_rate=0.5)

#n_estimators= 10
# Кросс-валидация проведена за: 0:01:06.490959
# 0.684251346261
# n_estimators= 20
# Кросс-валидация проведена за: 0:02:17.414851
# 0.697348323307
# n_estimators= 30
# Кросс-валидация проведена за: 0:03:22.130787
# 0.702367108827
# n_estimators= 40
# Кросс-валидация проведена за: 0:04:52.590472
# 0.70559514774

Количество деревьев = 10
Кросс-валидация проведена за: 0:00:40.312236
Качество: 0.684527487251
Количество деревьев = 20
Кросс-валидация проведена за: 0:01:21.503651
Качество: 0.697243670639
Количество деревьев = 30
Кросс-валидация проведена за: 0:02:30.854144
Качество: 0.702561905698
Количество деревьев = 40
Кросс-валидация проведена за: 0:02:30.849922
Качество: 0.706219091941


Были проведены тесты при значениях **n_folds**: 10, 20, 30, 40. 

С увеличением количества деревьев качество градиентного бустинга продолжает расти (максимальное проверенное значение - 40). Для ускорения работы был использован параметр **max_depth** = 2 (однако при ответе на вопрос задания была использована глубина по умолчанию).

Следовательно, имеет смысл использовать больше 30 деревьев, особенно при достаточных вычислительных мощностях.

Также имеет смысл уменьшить глубину деревьев или ограничиться лишь частью выборки (половиной или даже одной третью при огромных обучающих выборках), чтобы ускорить процесс обучения. Более этого оба этих метода можно совместить, подобрав оптимальные параметры опытным путём. И не стоит забывать, что значительное уменьшение выборки может привести к переобучению.

### Подход 2: логистическая регрессия

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV

kf = KFold(n=number_of_matches, n_folds=5, shuffle=True)
features_scaled = StandardScaler().fit_transform(features)

def find_best_param(data, target_col):
    global C
    grid = {"C": np.power(10.0, np.arange(-5, 6))}
    clf = LogisticRegression(penalty="l2", C=grid)
    gs = GridSearchCV(clf, grid,  scoring="roc_auc", cv=kf)
    gs.fit(data, target_col)
    C = gs.best_params_["C"]
    best_score = gs.best_score_
    print(C)
    print(best_score)
    
# find_best_param(features_scaled, target)  # C=0.01, score=0.71641485662

In [7]:
# Обучим классификатор отдельно для лучшего параметра и замерим время, которое для этого потребуется
def logistic_regression(data, target_col, C_best):
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty="l2", C=C_best)
    clf.fit(data, target_col)
    scores = cross_val_score(clf, data, target_col, cv=kf, scoring="roc_auc")
    mean_score = scores.mean()
    print("Качество:", mean_score)
    print('Кросс-валидация проведена за:', datetime.datetime.now() - start_time)  # 0:00:15.286640
    
# logistic_regression(features_scaled, target, C)  # 0.71641485662; 0:00:14.681357

In [8]:
# Избавимся от категориальных признаков
#  lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero
features_no_categories = features.drop(["lobby_type", "r1_hero", "r2_hero", "r3_hero", "r4_hero", "r5_hero",
                                      "d1_hero", "d2_hero", "d3_hero", "d4_hero", "d5_hero"], axis=1)
features_no_categories = StandardScaler().fit_transform(features_no_categories)

# find_best_param(features_no_categories, target)  # C=0.01, 0.716516975892

In [9]:
# Заново применим классификатор с лучшим параметром 

logistic_regression(features_no_categories, target, C) # Качество: 0.716516975892; Время: 0:00:13.347650

NameError: name 'C' is not defined

In [ ]:
# Посчитаем количество уникальных героев
match_heroes = ['{}{}_hero'.format(team, order) for team in ['r', 'd'] for order in range(1,6)]
hero_IDs = set()
for player in match_heroes:
    for hero in features[player]:
        hero_IDs.add(hero)

print("Количество уникальных идентификаторов:", len(hero_IDs))  # 108
N = max(hero_IDs)
print("Максимальный ID", N)  # 112

In [ ]:
# Добавление мешка слов
def calculate_bag_matrix(data, N):
    X_pick=np.zeros((data.shape[0], N))
    for i, match_id in enumerate(data.index):
        for p in range(5):
            X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)] - 1] = 1
            X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)] - 1] = -1
    return X_pick

bag = calculate_bag_matrix(features, N)

In [ ]:
# Расчёт для данных с мешком слов
features_no_categories_bag = np.hstack((features_no_categories, bag))
logistic_regression(features_no_categories_bag, target, C)  #0.751734076095; 0:00:20.653868

In [ ]:
# Приведём к аналогичному виду тестовые данные
features_test = pd.read_csv("./features_test.csv", index_col ="match_id")

number_of_matches = features_test.shape[0]
columns_full_check = np.array(features_test.count() == number_of_matches)
not_full = np.array(features_test.columns[columns_full_check == False])
for col in not_full:
    features_test[col].fillna(value=0, inplace=True)  # заменяем в этих колонках отсутствующие значения на 0
bag_test = calculate_bag_matrix(features_test, N)
features_no_categories_test = features_test.drop(["lobby_type", "r1_hero", "r2_hero", "r3_hero", "r4_hero", "r5_hero",
                                      "d1_hero", "d2_hero", "d3_hero", "d4_hero", "d5_hero"], axis=1)
features_no_categories_bag_test = np.hstack((features_no_categories_test, bag_test))
features_test_scaled = StandardScaler().fit_transform(features_no_categories_bag_test)

In [ ]:
# Обучим наш лучший классификатор (логистическая регрессия с параметром C=0.01) на всей выборке
clf = LogisticRegression(penalty="l2", C=C)
clf.fit(features_no_categories_bag, target)

# Применим его на тестовых данных
pred = clf.predict_proba(features_test_scaled)[:, 1]
print(pred.min())  # 8.71766397829e-05
print(pred.max())  # 0.999959812711